# Eksempel på bruk av PxWebApi versjon 2 beta i Python


## Det nye APIet tilbyr både http POST som før og http GET som er nytt

#### benytter biblioteket pyjstat for JSON-stat

In [1]:
import pandas as pd
import requests
from pyjstat import pyjstat 

## Post spørring og få Pandas dataframe i retur
Nytt: Språk og utformat angis i URL

In [2]:
# Eksport av laks
POST_URL = 'https://data.ssb.no/api/pxwebapi/v2-beta/tables/03024/data?lang=no&outputFormat=json-stat2'

API spørring, Det er en annen struktur enn i PxWebApi Versjon 1  - henter fryst og fersk laks siste 53 uker

In [5]:

payload = {"selection": [
			{"variableCode": "VareGrupper2", 
             "valueCodes": [
                 "01", "02"
             ] 
            },
			{"variableCode": "ContentsCode", 
             "valueCodes": [
                 "Vekt", "Kilopris"
             ] 
            },
			{"variableCode": "Tid", 
             "valueCodes": ["2025*"] }
		]
		}



In [6]:
resultat = requests.post(POST_URL, json = payload)

In [7]:
print(resultat)

<Response [200]>


Resultat gir bare http statuskode - 200 hvis OK. Body ligger i resultat.text

In [8]:
dataset = pyjstat.Dataset.read(resultat.text)

In [9]:
df = dataset.write('dataframe')

In [10]:
df.head()

,varegruppe,statistikkvariabel,uke,value
0,Fersk oppalen laks,Vekt (tonn),2025U01,11364.0
1,Fersk oppalen laks,Vekt (tonn),2025U02,15939.0
2,Fersk oppalen laks,Vekt (tonn),2025U03,17325.0
3,Fersk oppalen laks,Vekt (tonn),2025U04,15564.0
4,Fersk oppalen laks,Vekt (tonn),2025U05,16370.0


In [11]:
df.tail()

,varegruppe,statistikkvariabel,uke,value
119,Frosen oppalen laks,Kilopris (kr),2025U27,76.87
120,Frosen oppalen laks,Kilopris (kr),2025U28,77.98
121,Frosen oppalen laks,Kilopris (kr),2025U29,76.83
122,Frosen oppalen laks,Kilopris (kr),2025U30,70.76
123,Frosen oppalen laks,Kilopris (kr),2025U31,71.67


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   varegruppe          124 non-null    object 
 1   statistikkvariabel  124 non-null    object 
 2   uke                 124 non-null    object 
 3   value               124 non-null    float64
dtypes: float64(1), object(3)
memory usage: 4.0+ KB


In [13]:
print(resultat.text)

{"version":"2.0","class":"dataset","label":"03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke","source":"Statistisk sentralbyrå","updated":"2025-08-06T06:00:00Z","role":{"time":["Tid"],"metric":["ContentsCode"]},"id":["VareGrupper2","ContentsCode","Tid"],"size":[2,2,31],"dimension":{"VareGrupper2":{"label":"varegruppe","category":{"index":{"01":0,"02":1},"label":{"01":"Fersk oppalen laks","02":"Frosen oppalen laks"}},"extension":{"elimination":false,"show":"value"}},"ContentsCode":{"label":"statistikkvariabel","category":{"index":{"Vekt":0,"Kilopris":1},"label":{"Vekt":"Vekt (tonn)","Kilopris":"Kilopris (kr)"},"unit":{"Vekt":{"base":"tonn","decimals":0},"Kilopris":{"base":"kr","decimals":2}}},"extension":{"elimination":false,"refperiod":{"Vekt":"Utgangen av veka","Kilopris":"Utgangen av veka"},"show":"value","measuringType":{"Vekt":"Flow","Kilopris":"Flow"},"priceType":{"Vekt":"NotApplicable","Kilopris":"NotApplicable"},"adjustment":{"Vekt":"None","Kilopris":"

## Henter noen overordnete metadata fra JSON-stat datasettet

In [14]:
tittel = dataset['label']
print(tittel)

03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke


In [15]:
last_update = dataset['updated']
print(last_update)

2025-08-06T06:00:00Z


In [16]:
last_update = dataset['updated']
print(last_update)

2025-08-06T06:00:00Z


In [17]:
source = dataset['source']
print(source)

Statistisk sentralbyrå


'Role' gir snarveier til statistikkvariabel(metric), Tid og evt. Geo

In [18]:
ds_roles = dataset['role']
print(ds_roles)

OrderedDict({'time': ['Tid'], 'metric': ['ContentsCode']})


Her er noen nye metadata

In [19]:
ds_id = dataset['extension']['px']['tableid']
print(ds_id)

03024


In [20]:
ds_shorttit = dataset['extension']['px']['contents']
print(ds_shorttit)

03024: Eksport av oppalen laks,


Sjekk om tabellen har fotnote

In [21]:
if 'note' in dataset:
    ds_note = dataset['note']
    print(ds_note)

In [22]:
ds_dims = dataset['id']
ds_dims

['VareGrupper2', 'ContentsCode', 'Tid']

In [23]:
ds_dim = dataset['dimension']['VareGrupper2']['label']
ds_dim

'varegruppe'

In [24]:
ds_shorttit,' etter ',ds_dim

('03024: Eksport av oppalen laks,', ' etter ', 'varegruppe')

In [25]:
ds_dimcat = dataset['dimension']['ContentsCode']
ds_dimcat

OrderedDict([('label', 'statistikkvariabel'),
             ('category',
              OrderedDict([('index',
                            OrderedDict([('Vekt', 0), ('Kilopris', 1)])),
                           ('label',
                            OrderedDict([('Vekt', 'Vekt (tonn)'),
                                         ('Kilopris', 'Kilopris (kr)')])),
                           ('unit',
                            OrderedDict([('Vekt',
                                          OrderedDict([('base', 'tonn'),
                                                       ('decimals', 0)])),
                                         ('Kilopris',
                                          OrderedDict([('base', 'kr'),
                                                       ('decimals',
                                                        2)]))]))])),
             ('extension',
              OrderedDict([('elimination', False),
                           ('refperiod',
                        

In [26]:
ds_var = dataset['dimension']['VareGrupper2']['category']['label']['01']
ds_var

'Fersk oppalen laks'

## GET - PxWebApi2

Ferdigdefinerte datasett er nytt versjon 2 av PxWebApi. 

Vi bruker POST-URLen over med http GET til å gi oss et ferdigdefinert datasett.

Vi kunne tatt vekk parametrene ?lang=no&outputFormat=json-stat2 fordi disse er satt i /config

In [27]:
print(requests.get(POST_URL).url)

https://data.ssb.no/api/pxwebapi/v2-beta/tables/03024/data?lang=no&outputFormat=json-stat2


In [28]:
resultat_get = requests.get(POST_URL)

In [29]:
res_get = resultat_get.text

In [30]:
dataset_get = pyjstat.Dataset.read(res_get)

In [31]:
# dataset_get

In [32]:
df_get = dataset_get.write('dataframe')

In [33]:
df_get

,varegruppe,uke,statistikkvariabel,value
0,Fersk oppalen laks,2025U31,Vekt (tonn),25317.00
1,Fersk oppalen laks,2025U31,Kilopris (kr),60.58
2,Frosen oppalen laks,2025U31,Vekt (tonn),854.00
3,Frosen oppalen laks,2025U31,Kilopris (kr),71.67


Vi kan også bruke get-url med parametre: fersk laks, vekt, pris i 2025

`&valueCodes[VareGrupper2]=01&valueCodes[ContentsCode]=Vekt,Kilopris&valueCodes[Tid]=2025*`

In [34]:
GET_URL = 'https://data.ssb.no/api/pxwebapi/v2-beta/tables/03024/data?lang=no&valueCodes[VareGrupper2]=01&valueCodes[ContentsCode]=Vekt,Kilopris&valueCodes[Tid]=2025*'

In [35]:
resultat_get1 = requests.get(GET_URL)

In [36]:
dataset1_get = pyjstat.Dataset.read(resultat_get1.text)

In [37]:
df1 = dataset1_get.write('dataframe')

In [38]:
df1.tail(2)

,varegruppe,statistikkvariabel,uke,value
60,Fersk oppalen laks,Kilopris (kr),2025U30,65.10
61,Fersk oppalen laks,Kilopris (kr),2025U31,60.58


### Nytt i API2. Parametre for styring av CSV-output.
Dette er samme som POST spørringen, men med utformat CSV. 

Output her er semikolonseparert, med både kode og tekst. Statistikkvariabel i hode, rekkefølge på kolonner er Varegruppe2 og Tid

`outputformat = csv`

`outputformatparams = separatorsemicolon, usecodesandtexts`

`heading = ContentsCode`

`stub = VareGrupper2, Tid`

In [39]:
CSV_URL = 'https://data.ssb.no/api/pxwebapi/v2-beta/tables/03024/data?lang=no&outputformat=csv&outputformatparams=separatorsemicolon,usecodesandtexts&heading=ContentsCode&stub=VareGrupper2,Tid&valuecodes[ContentsCode]=Vekt,Kilopris&valuecodes[Varegrupper2]=01,02&valuecodes[Tid]=2025*'

Det er også mulig å angi URL med CSV direkte til Pandas og få en dataframe. 

In [40]:
df = pd.read_csv(CSV_URL, delimiter = ';')

In [41]:
df.head(8)

,VareGrupper2 - varegruppe,Tid - uke,Vekt - Vekt (tonn),Kilopris - Kilopris (kr)
0,01 - Fersk oppalen laks,2025U01 - 2025U01,11364,125.36
1,01 - Fersk oppalen laks,2025U02 - 2025U02,15939,117.77
2,01 - Fersk oppalen laks,2025U03 - 2025U03,17325,105.08
3,01 - Fersk oppalen laks,2025U04 - 2025U04,15564,98.43
4,01 - Fersk oppalen laks,2025U05 - 2025U05,16370,94.49
5,01 - Fersk oppalen laks,2025U06 - 2025U06,15184,92.18
6,01 - Fersk oppalen laks,2025U07 - 2025U07,15966,93.23
7,01 - Fersk oppalen laks,2025U08 - 2025U08,15536,92.97


Det samme, men mye mer omstendelig om en ikke bruker URL direkte

In [42]:
resultat_csv = requests.get(CSV_URL)

In [43]:
resultat_csv

<Response [200]>

In [44]:
r_csv = resultat_csv.text

In [45]:
r_csv

'"VareGrupper2 - varegruppe";"Tid - uke";"Vekt - Vekt (tonn)";"Kilopris - Kilopris (kr)"\r\n"01 - Fersk oppalen laks";"2025U01 - 2025U01";11364;125.36\r\n"01 - Fersk oppalen laks";"2025U02 - 2025U02";15939;117.77\r\n"01 - Fersk oppalen laks";"2025U03 - 2025U03";17325;105.08\r\n"01 - Fersk oppalen laks";"2025U04 - 2025U04";15564;98.43\r\n"01 - Fersk oppalen laks";"2025U05 - 2025U05";16370;94.49\r\n"01 - Fersk oppalen laks";"2025U06 - 2025U06";15184;92.18\r\n"01 - Fersk oppalen laks";"2025U07 - 2025U07";15966;93.23\r\n"01 - Fersk oppalen laks";"2025U08 - 2025U08";15536;92.97\r\n"01 - Fersk oppalen laks";"2025U09 - 2025U09";18067;92.67\r\n"01 - Fersk oppalen laks";"2025U10 - 2025U10";17116;87.00\r\n"01 - Fersk oppalen laks";"2025U11 - 2025U11";17467;88.64\r\n"01 - Fersk oppalen laks";"2025U12 - 2025U12";18172;89.09\r\n"01 - Fersk oppalen laks";"2025U13 - 2025U13";18828;84.45\r\n"01 - Fersk oppalen laks";"2025U14 - 2025U14";19053;76.40\r\n"01 - Fersk oppalen laks";"2025U15 - 2025U15";20009

In [46]:
import io
df1 = pd.read_csv(io.StringIO(r_csv), delimiter = ';')

In [47]:
df1.tail(8)

,VareGrupper2 - varegruppe,Tid - uke,Vekt - Vekt (tonn),Kilopris - Kilopris (kr)
54,02 - Frosen oppalen laks,2025U24 - 2025U24,651,75.44
55,02 - Frosen oppalen laks,2025U25 - 2025U25,688,73.33
56,02 - Frosen oppalen laks,2025U26 - 2025U26,440,72.93
57,02 - Frosen oppalen laks,2025U27 - 2025U27,628,76.87
58,02 - Frosen oppalen laks,2025U28 - 2025U28,657,77.98
59,02 - Frosen oppalen laks,2025U29 - 2025U29,606,76.83
60,02 - Frosen oppalen laks,2025U30 - 2025U30,718,70.76
61,02 - Frosen oppalen laks,2025U31 - 2025U31,854,71.67
